In [1]:
import config #file with client_id and secret for the Spotify API
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [26]:
list_of_playlists=["Top 100 Greatest Songs of All Time", "Best Soundtracks of all time", "huge rock rotation", "Pop Hits 2024", "Pop Party", "Rock Hits", "Rock Classics", "Big rock tracks from 2000", "Hip-Hop 2023", "Hip Hop Dance Playlist", "Hip-Hop Classics Party", "EDM House mix", "Eletronic Music Mix", "Electronic Dance Music", "Techno Music 2024", "Techno Remix", "Viva Latino", "Latino Dance Hits", "The 100 most streamed songs on Spotify", "RnB Hits 2024", "Jazz Classics", "Best Classical Music", "kids playlist 2023", "Hot hits Deutschland", "Happy Dance"]
len(list_of_playlists)

23

In [27]:
#def function fo extracting tracks from the playlists and place them into list

#function to get playlist_id

def playlist_id(name):
    result=sp.search(q=name,limit=5,type="playlist", market="GB")
    if result["playlists"]["items"]:
        playlist_id=result["playlists"]["items"][0]["id"]
        print(f"ID for {name} is found, ID:{playlist_id}")
        #return playlist_id
        return get_playlist_tracks ("Spotify", playlist_id)
    else:
        print("not found")

#function to get the list of tracks from specific playlist_id    
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return songs_ids(tracks)
       
#function to get a list of songs
def songs_ids(tracks):
    list_song_ids=[]
    for i in tracks:
        list_song_ids.append(i['track']['id'])
    return chunk(list_song_ids)

#function to do the chunk thing
def chunk(list_song_ids):
    chunk_size = 100
    list_chopped = [list_song_ids[i:i+chunk_size] for i in range(0,len(list_song_ids), 100)]
    return get_AF(list_chopped)

#function to get AF    
def get_AF(list_chopped):
    list_af = []
    for i in list_chopped:
        list_af.extend(sp.audio_features(i))
    df=pd.DataFrame(list_af)
    return df

In [28]:
#test all functions with the 1st playlist
todays_top_hits=playlist_id("today's top hits")

ID for today's top hits is found, ID:37i9dQZF1DXcBWIGoYBM5M


In [29]:
#creating an empty DF to fill
final_df=pd.DataFrame()

In [30]:
#running functions for each genre from the list

for playlist in list_of_playlists:
    df_temp=playlist_id(playlist)
    final_df=pd.concat([final_df, df_temp])

ID for Top 100 Greatest Songs of All Time is found, ID:6i2Qd6OpeRBAzxfscNXeWp
ID for Pop Hits 2024 is found, ID:5TDtuKDbOhrfW7C58XnriZ
ID for Pop Party is found, ID:2WwJVcVXsJP2dkfKVrOY4h
ID for Rock Hits is found, ID:3UNOpyCZYYtjd24JuFgXLM
ID for Rock Classics is found, ID:37i9dQZF1DWXRqgorJj26U
ID for Big rock tracks from 2000 is found, ID:37i9dQZF1DX6rsDrBNGuWW
ID for Hip-Hop 2023 is found, ID:0HpT5j5D5vHL0sPyecTKQr
ID for Hip Hop Dance Playlist is found, ID:5B46dEOFzUu8z3uPC9lBLt
ID for Hip-Hop Classics Party is found, ID:37i9dQZF1DXbkfWVLd8wE3
ID for EDM House mix is found, ID:37i9dQZF1EIdDyy28MYSyS
ID for Eletronic Music Mix is found, ID:0MJjDVvC20xinTETjoZrvM
ID for Electronic Dance Music is found, ID:6JTc4cdN83XTFgsUB3XLPA
ID for Techno Music 2024 is found, ID:18vUeZ9BdtMRNV6gI8RnR6
ID for Techno Remix is found, ID:6Q6nROUKNVXTu7UNpQXmzc
ID for Viva Latino is found, ID:37i9dQZF1DX10zKzsJ2jva
ID for Latino Dance Hits is found, ID:1amPv4WLmtqRrPUSzxADyG
ID for The 100 most stream

In [31]:
#dropping duplicates from the final DF
final_df=final_df.drop_duplicates()

In [32]:
#ready #checking NAs
print(final_df.isna().sum()>0)

final_df

danceability        False
energy              False
key                 False
loudness            False
mode                False
speechiness         False
acousticness        False
instrumentalness    False
liveness            False
valence             False
tempo               False
type                False
id                  False
uri                 False
track_href          False
analysis_url        False
duration_ms         False
time_signature      False
dtype: bool


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.647,0.582,10,-9.538,0,0.0755,0.41000,0.000002,0.0715,0.915,167.396,audio_features,4gphxUgq0JSFv2BCLhNDiE,spotify:track:4gphxUgq0JSFv2BCLhNDiE,https://api.spotify.com/v1/tracks/4gphxUgq0JSF...,https://api.spotify.com/v1/audio-analysis/4gph...,146480,4
1,0.579,0.508,2,-9.484,1,0.0270,0.00574,0.000494,0.0575,0.609,147.125,audio_features,40riOy7x9W7GXjyGp4pjAv,spotify:track:40riOy7x9W7GXjyGp4pjAv,https://api.spotify.com/v1/tracks/40riOy7x9W7G...,https://api.spotify.com/v1/audio-analysis/40ri...,391376,4
2,0.920,0.654,11,-3.051,0,0.0401,0.02360,0.015300,0.0360,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
3,0.414,0.404,0,-9.928,0,0.0499,0.27100,0.000000,0.3000,0.224,71.105,audio_features,4u7EnebtmKWzUH433cf5Qv,spotify:track:4u7EnebtmKWzUH433cf5Qv,https://api.spotify.com/v1/tracks/4u7EnebtmKWz...,https://api.spotify.com/v1/audio-analysis/4u7E...,354320,4
4,0.443,0.403,0,-8.339,1,0.0322,0.63100,0.000000,0.1110,0.410,143.462,audio_features,7iN1s7xHE4ifF5povM6A48,spotify:track:7iN1s7xHE4ifF5povM6A48,https://api.spotify.com/v1/tracks/7iN1s7xHE4if...,https://api.spotify.com/v1/audio-analysis/7iN1...,243027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.762,0.863,0,-3.689,0,0.0561,0.01500,0.000000,0.1250,0.965,106.012,audio_features,6s8nHXTJVqFjXE4yVZPDHR,spotify:track:6s8nHXTJVqFjXE4yVZPDHR,https://api.spotify.com/v1/tracks/6s8nHXTJVqFj...,https://api.spotify.com/v1/audio-analysis/6s8n...,185587,4
46,0.616,0.841,5,-4.779,0,0.0760,0.05590,0.002170,0.4170,0.714,110.029,audio_features,5YdnOm5990Kfq1Jodws98B,spotify:track:5YdnOm5990Kfq1Jodws98B,https://api.spotify.com/v1/tracks/5YdnOm5990Kf...,https://api.spotify.com/v1/audio-analysis/5Ydn...,154561,4
47,0.754,0.802,9,-6.424,1,0.0551,0.03490,0.000000,0.3350,0.583,122.026,audio_features,4Tt6GiWS2V9sQjZjS2xjp2,spotify:track:4Tt6GiWS2V9sQjZjS2xjp2,https://api.spotify.com/v1/tracks/4Tt6GiWS2V9s...,https://api.spotify.com/v1/audio-analysis/4Tt6...,199263,4
48,0.720,0.791,1,-4.689,1,0.1240,0.03840,0.000000,0.1570,0.756,102.071,audio_features,6FE2iI43OZnszFLuLtvvmg,spotify:track:6FE2iI43OZnszFLuLtvvmg,https://api.spotify.com/v1/tracks/6FE2iI43OZns...,https://api.spotify.com/v1/audio-analysis/6FE2...,175427,4


In [33]:
final_df.to_csv('songs_with_AF.csv', index=False)